### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [6]:
import pandas as pd
import numpy as np
import os
import re

In [72]:
pd.read_excel?

In [178]:
# Read xls file 
#peta_to_giga = lambda x: x*1000000 if ~np.isnan(x) else np.nan
#path = os.getcwd() + "\\Data\\Energy Indicators.xls"
path = os.getcwd() + "/Energy Indicators.xls"

with pd.ExcelFile(path) as con:
    df1 = pd.read_excel(io = path, sheetname = 'Energy',skiprows = 18, 
                        header = None, index_col = None,
                       names = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'],
                       dtype = {'Country' : 'str', 
                                'Energy Supply' : np.float64,  # will work even is NaN are there, as NaN is float
                                'Energy Supply per Capita' : np.float64,
                                '% Renewable' : np.float64},
                       # converters = {'Energy Supply' : peta_to_giga, },
                        parse_cols = "C:F", 
                        na_values = ['', 'NA', None, '...'], 
                        verbose = True)
df1['Energy Suppply(GJ)'] = df1['Energy Supply'].map(lambda x : x*1000000) 
df1.drop('Energy Supply', axis= 1,inplace = True)
df1.dropna(axis = 0, how = 'all',inplace = True) # drop empty rows

# Function to make Corrections 
def dic_corrections(x):
    '''
    1. Applies corrections stored in a dictionary to a string 
    2. Remove parts of string, if any in parentheses, and any numbers in the string
    
    '''
    cor_dic = {"South Korea" : [["Republic", 'Korea'], ["Rep.", 'Korea']],
            "United States" : [["United", "States", "America"], ],
            "United Kingdom" : [["United", "Kingdom", "Britain"], ["Northern", "Ireland"]] ,
            "Hong Kong"  : [['China',"Hong" ,"Kong"],],
            "Iran" : [["Iran",],]}
    
    lt_words = x.replace(',', '').split(sep = ' ')
    #print(lt_words)
    for k,vals in cor_dic.items():
        #print(k, vals)
        for lt in vals:
            check = all([el in lt_words for el in lt])
            #print(check)
            if(check & (k!= 'South Korea')):
                return(k)
            elif(check & (k== 'South Korea') & (len(lt_words) < 4)):
                return(k)
    return(x)

def other_corrections(x):
    # partiton on '(' and extract 1st component of the resulting list
    temp = re.search('\(', x) 
    if(temp is not None):
        result = x[0: temp.start()]
    else:
        result = x
    #print(result)    
    m = re.search('[0-9]+', result)  
    if(m is not None):
        result_fin = result.replace(result[m.start(): m.end()], '')
    else:
        result_fin = result
        #print(result)
    return result_fin.strip()
    
def correct(x):
    temp = other_corrections(x)
    return(dic_corrections(temp))
            
    
# Get data from web 
# Get data from web

Reading sheet Energy


In [179]:
correct('China, Hong Kong Special Administrative Region3')

'Hong Kong'

In [180]:
df1['Country_new'] = df1['Country'].map(correct)

In [97]:
#df1.set_index('Country_new',inplace= True)

In [102]:
df1.loc[df1['Country_new'].map(lambda x : x.startswith('Iran')),]

,Country,Energy Supply per Capita,% Renewable,Energy Suppply(GJ),Country_new
98,Iran (Islamic Republic of),119.0,5.707721,9.172000e+09,Iran


In [181]:
df1.loc[df1['Country_new'] != df1['Country'],]

,Country,Energy Supply per Capita,% Renewable,Energy Suppply(GJ),Country_new
11,Australia1,231.0,11.810810,5.386000e+09,Australia
24,Bolivia (Plurinational State of),32.0,31.477120,3.360000e+08,Bolivia
42,China2,93.0,19.754910,1.271910e+11,China
43,"China, Hong Kong Special Administrative Region3",82.0,0.000000,5.850000e+08,Hong Kong
44,"China, Macao Special Administrative Region4",83.0,0.000000,4.700000e+07,"China, Macao Special Administrative Region"
58,Denmark5,129.0,33.537660,7.250000e+08,Denmark
70,Falkland Islands (Malvinas),258.0,33.333330,1.000000e+06,Falkland Islands
73,France6,166.0,17.020280,1.059700e+10,France
83,Greenland7,166.0,78.125000,9.000000e+06,Greenland
97,Indonesia8,35.0,12.189040,8.754000e+09,Indonesia
